# Conversational Interface - Contextual-aware Chatbot with Strands SDK and RAG

In this notebook we will build a chatbot using Strands SDK that automatically handles conversation history and uses context from documents through RAG (Retrieval Augmented Generation). Unlike traditional approaches that require manual session management with a database, Strands provides built-in conversation tracking.

## Key Advantages of Strands SDK:
- **Built-in conversation management** - No need for a database setup
- **Automatic context preservation** - Maintains conversation state across interactions
- **Simplified agent creation** - Less boilerplate code
- **Custom tool integration** - Easy to add RAG capabilities with custom tools

In [ ]:
import boto3
import json
import pprint
from typing import List
from strands import tool, Agent

pp = pprint.PrettyPrinter(indent=2)

import sys
sys.path.append('../')
from util.model_selector import create_text_model_selector, create_embedding_model_selector

# Create interactive model selector
model_selector = create_text_model_selector().display()
bedrock_model = model_selector.get_model_id()
print("\n🎯 Select your preferred model above and run the cells below to see it in action!")

### Set up

In [ ]:
boto3_session = boto3.session.Session()
region = boto3_session.region_name or "us-east-1"

# Get the selected model
selected_model = model_selector.get_model_id()
model_info = model_selector.get_model_info()

print(f"Using model: {model_info['name']} ({selected_model})")
print(f"Description: {model_info['description']}")

temperature = 0.1

## A simple local vector store to demonstrate the RAG pattern with Strands SDK

The RAG pattern enhances Q&A systems by retrieving relevant document chunks before generating responses. When a user asks a question, the system performs a similarity search against a vector database containing document embeddings, retrieves the most relevant chunks, and includes them as context in the LLM prompt to generate more accurate, grounded answers.

With Strands SDK, we can create custom tools that handle document retrieval and integrate them seamlessly with the agent's conversation capabilities.

First let's load the 2022 Shareholder letter from Andy Jassy (pdf) and split it into chunks

In [ ]:
# Document processing (same as before, but we'll use it with Strands tools)
from PyPDF2 import PdfReader
import re

def load_and_split_pdf(file_path: str, chunk_size: int = 1000, chunk_overlap: int = 200) -> List[str]:
    """Load PDF and split into chunks"""
    # Read PDF
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    
    # Simple text splitting
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        if end < len(text):
            # Try to break at a sentence or word boundary
            while end > start and text[end] not in '.!?\n ':
                end -= 1
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        start = end - chunk_overlap if end < len(text) else end
    
    return chunks

file_name = "./data/AMZN-2022-Shareholder-Letter.pdf"
document_chunks = load_and_split_pdf(file_name)

print(f"Loaded {len(document_chunks)} chunks from the PDF")
print("First chunk preview:")
pp.pprint(document_chunks[0][:500] + "...")

Now we will create an in-memory FAISS db to hold the chunks from the previous step, which we will use to retrieve the relevant context (RAG pattern) based on the user query

In [ ]:
# Create interactive embedding model selector
embedding_model_selector = create_embedding_model_selector().display()

print("\n🎯 Select your preferred embedding model above and run the cells below to see it in action!")

In [ ]:
import faiss
import numpy as np

# Get the selected embedding model
selected_embedding_model = embedding_model_selector.get_model_id()
embedding_model_info = embedding_model_selector.get_model_info()

print(f"Using embedding model: {embedding_model_info['name']} ({selected_embedding_model})")

# Create embeddings for document chunks
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

def get_embedding(text: str) -> np.ndarray:
    """Get embedding for a text using Bedrock"""
    response = bedrock_client.invoke_model(
        modelId=selected_embedding_model,
        body=json.dumps({"inputText": text})
    )
    response_body = json.loads(response['body'].read())
    return np.array(response_body['embedding'])

# Create embeddings for all chunks
print("Creating embeddings for document chunks...")
embeddings = []
for i, chunk in enumerate(document_chunks):
    if i % 10 == 0:
        print(f"Processing chunk {i}/{len(document_chunks)}")
    embedding = get_embedding(chunk)
    embeddings.append(embedding)

# Create FAISS index
embeddings_array = np.array(embeddings).astype('float32')
dimension = embeddings_array.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_array)

print(f"Created FAISS index with {index.ntotal} vectors of dimension {dimension}")

Now let's create a custom retrieval tool for Strands that will search the document chunks and provide context for answering questions.

In [ ]:
@tool
def document_retriever(query: str) -> str:
    """
    Search through Amazon's 2022 Shareholder Letter to find relevant information.
    This tool helps answer questions about Amazon's business, performance, and strategy.
    """
    # Get embedding for the query
    query_embedding = get_embedding(query).reshape(1, -1).astype('float32')
    
    # Search for similar chunks
    k = 3  # Number of chunks to retrieve
    distances, indices = index.search(query_embedding, k)
    
    # Get the relevant chunks
    relevant_chunks = []
    for i, idx in enumerate(indices[0]):
        if idx < len(document_chunks):
            relevant_chunks.append(f"Context {i+1}: {document_chunks[idx]}")
    
    return "\n\n".join(relevant_chunks)


# Test the retrieval tool
test_query = "What is Graviton?"
retrieved_context = document_retriever(test_query)
print("Retrieved context for 'What is Graviton?':")
print(retrieved_context[:1000] + "...")

### Create a Strands Agent with RAG capabilities

Now we'll create a conversational agent that can both maintain conversation history and retrieve relevant information from documents. This combines the best of both worlds - contextual conversation and knowledge retrieval.

In [ ]:
# Create a conversational RAG agent with Strands
rag_agent = Agent(
    system_prompt="""You are an assistant for question-answering tasks about Amazon's business and strategy. 
    Use the document_retriever tool to search for relevant information from Amazon's 2022 Shareholder Letter when answering questions.
    
    When a user asks a question:
    1. Use the document_retriever tool to find relevant context
    2. Answer the question based on the retrieved information
    3. If you don't know the answer even after searching, say that you don't know
    4. Keep answers concise (3 sentences maximum) but informative
    5. Maintain conversation context - if a follow-up question refers to previous topics, understand the connection
    
    You automatically maintain conversation history, so users can ask follow-up questions that reference previous topics.""",
    tools=[document_retriever],
    model=bedrock_model,
    callback_handler=None,  # default is PrintingCallbackHandler
)

print("RAG Agent created with built-in conversation history and document retrieval!")

### Test the conversational RAG agent

Now let's test our agent with questions that demonstrate both its retrieval capabilities and conversation memory.

In [ ]:
# Test the RAG agent with a question about Graviton
question1 = "What is Graviton?"
answer1 = rag_agent(question1)
print(f"Question: {question1}")
print(f"Answer: {answer1}")
print("\n" + "="*50 + "\n")

In [ ]:
# Test conversation memory with a follow-up question
# The agent should understand that "they" refers to Graviton processors from the previous question
question2 = "How much better price-performance do they deliver?"
answer2 = rag_agent(question2)
print(f"Question: {question2}")
print(f"Answer: {answer2}")
print("\n" + "="*50 + "\n")

In [ ]:
# Test with another topic
question3 = "What are Amazon's key investments in AWS?"
answer3 = rag_agent(question3)
print(f"Question: {question3}")
print(f"Answer: {answer3}")
print("\n" + "="*50 + "\n")

In [ ]:
# Test conversation context with a follow-up
question4 = "How do these investments compare to the previous year?"
answer4 = rag_agent(question4)
print(f"Question: {question4}")
print(f"Answer: {answer4}")